In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from pyclustering.cluster.kmedians import kmedians
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster import cluster_visualizer_multidim
from sklearn.decomposition import PCA
import pickle
from tqdm.notebook import tqdm

In [3]:
population_data_raw = pd.read_csv('../Data/population.csv')
more_than_50_raw = pd.read_csv('../Data/more_than_50k.csv')

In [4]:
def preprocessing(df):

    #replace ? with Nan
    df = df.replace({' ?' :  np.nan})

    #drop columns with more than 40$ NaN values
    cols = df.columns
    missing_percentages = (df.isna().sum() / len(df)) * 100
    cols_to_drop = []
    for col in cols:
        if(missing_percentages[col] > 40.0):
            cols_to_drop.append(col)
    print(f'Dropping = {cols_to_drop}')
    df = df.drop(cols_to_drop, axis=1)
    return df

In [5]:
data = preprocessing(population_data_raw)
print(f'Population data shape: {data.shape}')
data_more_than_50 = preprocessing(more_than_50_raw)
print(f'More than 50k data shape: {data_more_than_50.shape}')

Dropping = ['MIGMTR1', 'MIGMTR3', 'MIGMTR4', 'MIGSUN']
Population data shape: (199523, 36)
Dropping = ['MIGMTR1', 'MIGMTR3', 'MIGMTR4', 'MIGSUN']
More than 50k data shape: (3580, 36)


In [6]:
def plot_histogram(df, numeric_columns, anum):
    # features = df.columns
    # for feat in features:
    #     plt.figure(figsize=(25,5))
    #     if(feat in numeric_columns):
    #         df[feat].hist(bins=10,edgecolor='black', linewidth=1.5)

    #     else:
    #         unique_cat_values = len(df[feat].unique())
    #         print(f'No. of unique Categories : {unique_cat_values}')
    #         df[feat].hist(bins = unique_cat_values, edgecolor='black', linewidth=1.5)
    #     plt.xlabel(feat)
    #     plt.ylabel('Count')
    #     plt.title(f'Histogram : {feat}')
    #     plt.savefig(f'../Plots/Feature_Analysis/{feat}_hist', facecolor='w', bbox_inches='tight')
    #     plt.show()
    features = df.columns
    for feat in features:
        plt.figure(figsize=(25, 5))
        if(feat in numeric_columns):
            print('Numeric Column')
            df[feat].astype(str).hist(bins = 6, edgecolor='black', linewidth=1.5)
            
        else:
            unique_cat_values = len(df[feat].unique())
            print(f'Categorical Column || No. of unique Categories : {unique_cat_values}')
            df[feat].astype(str).hist(bins = unique_cat_values, edgecolor='black', linewidth=1.5)
        plt.xlabel(feat)
        plt.ylabel('Count')
        plt.title(f'Histogram : {feat}')
        plt.savefig(f'../Plots/Feature_Analysis_{anum}/{feat}_hist', facecolor='w', bbox_inches='tight')
        plt.show()

In [7]:
def feature_analysis(df, anum):
    data_columns = df.columns
    numeric_columns = ['AAGE', 'AHRSPAY', 'CAPGAIN', 'CAPLOSS', 'DIVVAL', 'WKSWORK']
    df_copy = df.copy(deep=True)
    for col in numeric_columns:
        df_copy[col] = pd.cut(x=df_copy[col], bins=6)
    # plot_histogram(df_copy, numeric_columns, anum)
    total_count = df_copy.shape[0]
    cols_to_drop = []
    for col in df_copy.columns:
        max_freq = df_copy[col].value_counts().max()
        max_perc = (max_freq / total_count) * 100
        if(max_perc > 80.0):
            cols_to_drop.append(col)
    print(f'{anum} : Dropping {len(cols_to_drop)} cols \n {cols_to_drop}')
    df = df.drop(cols_to_drop, axis=1)
    return df, cols_to_drop

    # print(df)
    # print(categorical_cols)


In [1]:
data, cols_to_drop_1 = feature_analysis(data, 1)
data_more_than_50 = data_more_than_50.drop(columns=cols_to_drop_1)
# data_more_than_50, cols_to_drop_2 = feature_analysis(data_more_than_50, 2)

NameError: name 'feature_analysis' is not defined

In [ ]:
# column_modes_1 = data.mode()
# column_modes_1

In [ ]:
def column_modifications(df):
    column_modes = df.mode()
    data_cols = df.columns
    numeric_columns = ['AAGE', 'AHRSPAY', 'CAPGAIN', 'CAPLOSS', 'DIVVAL', 'WKSWORK']
    for col in data_cols:
        missing_vals = df[col].isna().sum()
        if(missing_vals > 0):
            df[col] = df[col].fillna(column_modes[col][0])
    for col in df.columns:
        if(col in numeric_columns):
            df[col] = pd.cut(x=df[col], bins=6)
    one_hot_encoded = pd.get_dummies(df, columns=list(df.columns))
    df = one_hot_encoded
    return df

In [ ]:
data = column_modifications(data)
print(data.shape)
data_more_than_50 = column_modifications(data_more_than_50)
print(data_more_than_50.shape)

PCA 

In [ ]:
def performPCA(df, anum):
    pca = PCA()
    pca.fit(np.array(df))
    exp_var_rat = pca.explained_variance_ratio_
    cum_exp_var_perc = np.cumsum(exp_var_rat)
    plt.plot(cum_exp_var_perc)
    plt.xlabel('Cumulative explained variance')
    plt.ylabel('Number of components')
    plt.title(f'PCA Analysis - {anum}')
    plt.savefig(f'../Plots/Dimensionality_Reduction/pca_plot_{anum}.png', facecolor='white', bbox_inches='tight')
    plt.show()

    return pca, cum_exp_var_perc   

In [ ]:
pca_1, cum_var_1 = performPCA(data, 1)
pca_2, cum_var_2 = performPCA(data_more_than_50, 2)

In [ ]:
pca_final_1 = PCA(n_components=40)
dim_reduced_data_1 = pca_final_1.fit_transform(np.array(data))

pca_final_2 = PCA(n_components=40)
dim_reduced_data_2 = pca_final_2.fit_transform(np.array(data_more_than_50))

print(f' 1 - {dim_reduced_data_1.shape}')
print(f' 2 - {dim_reduced_data_2.shape}')

CLUSTERING

In [ ]:
num_samples_1 = data.shape[0]
num_samples_2 = data_more_than_50.shape[0]
print(num_samples_1, num_samples_2)

In [ ]:
def compute_avg_within_cluster_dist(dim_red_data, clust, centroids, k):
    total_sum = 0
    for i in range(len(clust)):
        clust_dist = 0
        # print(f'i = {i} | len = {len(clust)}')
        for idx in clust[i]:
            point = np.array(dim_red_data[idx])
            dist_val = np.sqrt(np.sum(np.square(point - centroids[i])))
            clust_dist += dist_val
        total_sum += clust_dist / len(clust[i])
    return total_sum / k

In [ ]:
def plot_clust_dist(avg_clust_distance, anum):
    plt.plot([x for x in range(10, 25)], list(avg_clust_distance.values()), marker='o')
    plt.xlabel('No. of clusters')
    plt.ylabel('Average cluster distance')
    plt.title(f'avg cluster distance vs number of clusters-{anum}')
    plt.savefig(f'../Plots/cluster_distance_{anum}', facecolor='white', bbox_inches='tight')
    plt.show()

In [ ]:
avg_cluster_distance_byK_1 = {}
avg_cluster_distance_byN_1 = {}
# avg_within_cluster_distance_1 = {}
for k_value in tqdm(range(10, 25)):

    print(f'-------- K VALUE = {k_value} --------')
    random_points_1 = np.array(pd.DataFrame(dim_reduced_data_1).sample(n=k_value)).tolist()

    # print('PERFORMING CLUSTERING')
    medians_init_1 = random_points_1
    kmedians_clusterer_1 = kmedians(dim_reduced_data_1, medians_init_1)
    kmedians_clusterer_1.process()

    file = open(f'../models/Part_1/kmedians_{k_value}.pickle', 'wb')
    pickle.dump(kmedians_clusterer_1, file)
    file.close()

    clusters_1 = kmedians_clusterer_1.get_clusters()
    medians_1 = kmedians_clusterer_1.get_medians()
    print(f'len cluster = {len(clusters_1)} || len medians = {len(medians_1)}')
    np.save(f'../model_info/Part_1/clusters_{k_value}.npy', np.array(clusters_1))
    np.save(f'../model_info/Part_1/medians_{k_value}.npy', np.array(medians_1))

    avg_dist_byK_1 = kmedians_clusterer_1.get_total_wce() / k_value
    avg_dist_byN_1 = kmedians_clusterer_1.get_total_wce() / num_samples_1
    # avg_within_clust_dist_1 = compute_avg_within_cluster_dist(dim_reduced_data_1, clusters_1, medians_1, k_value)
    
    avg_cluster_distance_byK_1[k_value] = avg_dist_byK_1
    avg_cluster_distance_byN_1[k_value] = avg_dist_byN_1
    # avg_within_cluster_distance_1[k_value] = avg_within_clust_dist_1
    print(f'average within cluster distance (byK) = {avg_dist_byK_1}')
    print(f'average within cluster distance (byN) = {avg_dist_byN_1}')
    # print(f'average within cluster distance = {avg_within_clust_dist_1}')
    # print('INSTANCE PROCESSED')



In [ ]:
plot_clust_dist(avg_cluster_distance_byN_1, 1)

In [ ]:
plt.plot([x for x in range(10, 25)], list(avg_cluster_distance_byK_1.values()), marker='o')
plt.xlabel('No. of clusters')
plt.ylabel('Average cluster distance')
plt.title('avg cluster distance vs number of clusters(byK) (1)')
# plt.savefig(f'../Plots/cluster_distance_{anum}', facecolor='white', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot([x for x in range(10, 25)], list(avg_cluster_distance_byN_1.values()), marker='o')
plt.xlabel('No. of clusters')
plt.ylabel('Average cluster distance')
plt.title('avg cluster distance vs number of clusters(byN) (1)')
# plt.savefig(f'../Plots/cluster_distance_{anum}', facecolor='white', bbox_inches='tight')
plt.show()

More than 50k

In [ ]:
avg_cluster_distance_byK_2 = {}
avg_cluster_distance_byN_2 = {}
avg_within_cluster_distance_2 = {}

for k_value in tqdm(range(10, 25)):
    
    print(f'-------- K VALUE = {k_value} --------')
    # random_points_2 = np.array(pd.DataFrame(dim_reduced_data_2).sample(n=k_value)).tolist()
    random_points_2 = np.random.randn(k_value, 40)
    # print(random_points_2.shape)
    # random_points_2 = np.zeros((k_value, 40))
    
    # print('PERFORMING CLUSTERING')
    medians_init_2 = random_points_2
    print(np.array(medians_init_2).shape)
    kmedians_clusterer_2 = kmedians(dim_reduced_data_2, medians_init_2)
    kmedians_clusterer_2.process()

    file = open(f'../models/Part_2/kmedians_{k_value}.pickle', 'wb')
    pickle.dump(kmedians_clusterer_2, file)
    file.close()

    clusters_2 = kmedians_clusterer_2.get_clusters()
    medians_2 = kmedians_clusterer_2.get_medians()
    print(np.array(clusters_2).shape, np.array(medians_2).shape)
    np.save(f'../model_info/Part_2/clusters_{k_value}.npy', np.array(clusters_2))
    np.save(f'../model_info/Part_2/medians_{k_value}.npy', np.array(medians_2))

    avg_dist_byK_2 = kmedians_clusterer_2.get_total_wce() / k_value
    avg_dist_byN_2 = kmedians_clusterer_2.get_total_wce() / num_samples_2
    avg_within_clust_dist_2 = compute_avg_within_cluster_dist(dim_reduced_data_2, clusters_2, medians_2, k_value)

    avg_cluster_distance_byK_2[k_value] = avg_dist_byK_2
    avg_cluster_distance_byN_2[k_value] = avg_dist_byN_2
    avg_within_cluster_distance_2[k_value] = avg_within_clust_dist_2
    print(f'average within cluster distance (byK) = {avg_dist_byK_2}')
    print(f'average within cluster distance (byN) = {avg_dist_byN_2}')
    print(f'average within cluster distance = {avg_within_clust_dist_2}')
    # print('INSTANCE PROCESSED')



In [ ]:
plot_clust_dist(avg_within_cluster_distance_2, 2)

In [ ]:
plt.plot([x for x in range(10, 25)], list(avg_cluster_distance_byK_2.values()), marker='o')
plt.xlabel('No. of clusters')
plt.ylabel('Average cluster distance')
plt.title('avg cluster distance vs number of clusters(byK) (1)')
# plt.savefig(f'../Plots/cluster_distance_{anum}', facecolor='white', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot([x for x in range(10, 25)], list(avg_cluster_distance_byN_2.values()), marker='o')
plt.xlabel('No. of clusters')
plt.ylabel('Average cluster distance')
plt.title('avg cluster distance vs number of clusters(byN) (1)')
# plt.savefig(f'../Plots/cluster_distance_{anum}', facecolor='white', bbox_inches='tight')
plt.show()